https://adventofcode.com/2022/day/15

In [1]:
import re
from itertools import product

import numpy as np
from toolz import concat
from intervaltree import Interval, IntervalTree

In [2]:
with open("data/15.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [4]:
class Sensor:
    def __init__(self, x, y, bx, by):
        self.x, self.y, self.bx, self.by = x, y, bx, by
        mnhtn = self.mnhtn = abs(x - bx) + abs(y - by)
        self.top = (x, y - mnhtn)
        self.bottom = (x, y + mnhtn)
        self.left = (x - mnhtn, y)
        self.right = (x + mnhtn, y)

    def contains(self, x, y):
        return abs(x - self.x) + abs(y - self.y) <= self.mnhtn

    def x_interval_at_y(self, y):
        if self.top[1] <= y <= self.bottom[1]:
            d = self.mnhtn - abs(y - self.y)
            return Interval(self.x - d, self.x + d + 1)

    def __repr__(self):
        return f"<Sensor ({self.x}, {self.y}) ({self.bx, self.by})>"

In [5]:
def load_data(data, cls=Sensor):
    return [
        cls(*[int(n) for n in re.findall(r"[+-]?\d+", line)])
        for line in data.strip().splitlines()
    ]

In [6]:
sensors = load_data(testdata)
y_to_check = 10

itree = IntervalTree(
    iv for iv in (s.x_interval_at_y(y_to_check) for s in sensors) if iv is not None
)
itree.merge_overlaps()
covered_point_count = sum(x.length() for x in itree)
beacon_xs = {s.bx for s in sensors if s.by == y_to_check}
beacon_count = sum(1 for b in beacon_xs if any(iv.contains_point(b) for iv in itree))
covered_point_count - beacon_count

26

Part 1

In [7]:
%%time
sensors = load_data(data)
y_to_check = 2_000_000

itree = IntervalTree(
    iv for iv in (s.x_interval_at_y(y_to_check) for s in sensors) if iv is not None
)
itree.merge_overlaps()
covered_point_count = sum(x.length() for x in itree)
beacon_xs = {s.bx for s in sensors if s.by == y_to_check}
beacon_count = sum(1 for b in beacon_xs if any(iv.contains_point(b) for iv in itree))
covered_point_count - beacon_count

CPU times: user 758 µs, sys: 0 ns, total: 758 µs
Wall time: 767 µs


5564017

Part 2

In [8]:
class Sensor2(Sensor):
    def x_interval_at_y_clipped(self, y, clipmin=0, clipmax=4_000_000):
        if not self.top[1] <= y <= self.bottom[1]:
            return None
        if self.right[0] < clipmin or self.left[0] > clipmax:
            return None
        d = self.mnhtn - abs(y - self.y)
        return Interval(max(clipmin, self.x - d), min(clipmax + 1, self.x + d + 1))

Any point not covered must be near an intersection of exclusion zones -- within a couple of rows either way.
So instead of 4 million y values, we will only look at those for intersections += 2

In [22]:
def segment_intersection(s1, s2):
    a1, b1, c1 = linear_equation_coefficients(*s1)
    a2, b2, c2 = linear_equation_coefficients(*s2)
    try:
        x, y = np.linalg.solve([[a1, b1], [a2, b2]], [c1, c2])
    except np.linalg.LinAlgError:
        return
    ((s1x1, s1y1), (s1x2, s1y2)) = s1
    ((s2x1, s2y1), (s2x2, s2y2)) = s1
    if (
        between(x, s1x1, s1x2)
        and between(x, s2x1, s2x2)
        and between(y, s1y1, s1y2)
        and between(y, s2y1, s2y2)
    ):
        return (x, y)


def linear_equation_coefficients(a, b):
    ((x1, y1), (x2, y2)) = (a, b)
    m = (y2 - y1) / (x2 - x1)
    return m, -1, (m * x1) - y1


def between(a, b, c):
    if b > c:
        (b, c) = (c, b)
    return b <= a <= c

In [24]:
%%time
sensors = load_data(data, cls=Sensor2)
pos_segs = concat(((s.left, s.top), (s.bottom, s.right)) for s in sensors)
neg_segs = concat(((s.top, s.right), (s.left, s.bottom)) for s in sensors)
segprod = product(pos_segs, neg_segs)
intersections = (
    x for x in (segment_intersection(s1, s2) for (s1, s2) in segprod) if x is not None
)
ys = concat(
    range(int(y - 2), int(y + 2)) for x, y in intersections if 2 <= y <= 3_999_999
)

for y in ys:
    itree = IntervalTree(
        iv
        for iv in (s.x_interval_at_y_clipped(y, clipmax=4_000_000) for s in sensors)
        if iv is not None
    )
    itree.merge_overlaps()
    covered_count = sum(iv.length() for iv in itree)
    if covered_count < 4_000_001:
        print(y, len(itree), covered_count, itree)
        break

3398893 2 4000000 IntervalTree([Interval(0, 2889605), Interval(2889606, 4000001)])
CPU times: user 265 ms, sys: 0 ns, total: 265 ms
Wall time: 268 ms


In [11]:
(2889605 * 4_000_000) + 3398893

11558423398893